In [30]:
import os
import torch
import random
import numpy as np
import jason

In [63]:
def loadmodel(name):
    x = open('model_parameters/'+ name+'.json')
    data = json.load(x)
    model_structure = data['model_structure']
    model_parameters_names = data['model_parameters_names']
    model_parameters_vals = data['model_parameters_vals']
    model_parameters_names = [f'[{name.split(".")[0]}].{name.split(".")[1]}'
                                   for name in model_parameters_names.split('|')]
    received_model = nn.Sequential(
    *[eval('nn.' + layer) for layer in model_structure.split('|')]
    )
    for parameters_name, parameters_val in zip(model_parameters_names, model_parameters_vals):
        exec(f'received_model{parameters_name} = nn.Parameter(torch.tensor(parameters_val))')
    return received_model

In [65]:
def loadmodels(name):
    all=[]
    for i in name:
        all.append(loadmodel(i))
    return all


In [ ]:
img_path = dataset_path + '/images'